# Particle discrimination of simulated electron data in Geant4 CNPTEPC 
## Andrei R. Hanu - andrei.hanu@nasa.gov

In [1]:
# Matplotlib - 2D plotting library
from matplotlib import pyplot as plt
from matplotlib import rcParams

# Show matplotlib figures inline the notebook
%matplotlib inline

# Pandas - High-performance data analysis
import pandas

# Glob - Unix style pathname pattern expansion
# Used to aggregate folders and files into a single list so they can be iterated over
import glob

# Numpy - For handling of large, multi-dimensional arrays and matrices
import numpy as np

# Color palette library for Python
# How to choose a colour scheme for your data:
# http://earthobservatory.nasa.gov/blogs/elegantfigures/2013/08/05/subtleties-of-color-part-1-of-6/
import brewer2mpl

In [2]:
##########################################################################################
# Setting rcParams for publication quality graphs
fig_width_pt = 246.0                    # Get this from LaTeX using \showthe\columnwidth
inches_per_pt = 1.0/72.27               # Convert pt to inch
golden_mean = (np.sqrt(5)-1.0)/2.0      # Aesthetic ratio
fig_width = fig_width_pt*inches_per_pt  # Width in inches
fig_height = fig_width*golden_mean      # Height in inches
fig_size =  [fig_width, fig_height]
# fig_size =  [7.3,4.2]
fig_size =  [7.3*1.25,4.2*1.25]
# fig_size =  [7.3*1.5,4.2*1.5]
# fig_size =  [7.3*2,4.2*2]
params = {'backend': 'pdf',
        'axes.labelsize': 10,
        'legend.fontsize': 9,
        'xtick.labelsize': 10,
        'ytick.labelsize': 10,
        'xtick.major.size': 6,
        'xtick.major.width': 1,
        'xtick.minor.size': 2.5,
        'xtick.minor.width': 1,
        'ytick.major.size': 6,
        'ytick.major.width': 1,
        'ytick.minor.size': 2.5,
        'ytick.minor.width': 1,
        'font.family': 'sans-serif',
        'font.sans-serif': 'Avant Garde',
        'font.size': 10,
        'text.usetex': True,
        'figure.figsize': fig_size}

# Update rcParams
rcParams.update(params)

In [3]:
##########################################################################################
# Energy of incident particles (Electrons)
InputEnergies = np.array([np.linspace(1000,9000, num=9, endpoint=True, dtype = int),
                np.linspace(10000,90000, num=9, endpoint=True, dtype = int)
                ]).flatten()

# Folder containing simulation data
DataFolder = '../Results/ACD_4p0mm/Electrons/'

##########################################################################################
# Energy thresholds (in eV) for events that will be included in the analysis
Thr_TEPC = 100.
Thr_ACD = 100.

In [4]:
##########################################################################################
# Load the simulation data
DataFrame = []

##########################################################################################
# Data location folder
# Data filename template
DataFolder = '../Results/ACD_4p0mm/Electrons/'
DataFilename = 'e-_100000keV_Nr_100000000_ISO_nt_G4CNPTEPC_t*.csv'

##########################################################################################
# Process the simulation data files
# Column 0 - Energy deposited in the TEPC sensitive volume [eV]
# Column 1 - Track length of particles that entirely traverse the TEPC [mm]
# Column 2 - Energy deposited in the plastic scintillator [eV]
for dataFile in glob.glob(DataFolder + DataFilename):  
    print 'Importing data file: ' + dataFile
    df = pandas.read_csv(dataFile, header=-1, skiprows=6, usecols=[0,1,2])
    DataFrame.append(df) 
    
# Merge the DataFrames from each file
Data = pandas.concat(DataFrame).values

# Print the number of total events
print '\nTotal number of events: \t\t', Data[:,0].size
print 'Number of coincidence events: \t\t', Data[np.logical_and((Data[:,2] > Thr_ACD),(Data[:,0] > Thr_TEPC)),0].size
print 'Number of anti-coincidence events: \t', Data[np.logical_and((Data[:,2] <= Thr_ACD),(Data[:,0] > Thr_TEPC)),0].size
print 'Number of sub-threshold events: \t', Data[np.logical_and((Data[:,2] <= Thr_ACD),(Data[:,0] <= Thr_TEPC)),0].size

ValueError: No objects to concatenate

In [ ]:
eDep_TEPC = Data[np.logical_and((Data[:,2] > Thr_ACD),(Data[:,0] > Thr_TEPC)),0]
eDep_ACD = Data[np.logical_and((Data[:,2] > Thr_ACD),(Data[:,0] > Thr_TEPC)),2]
print np.min(eDep_TEPC),np.median(eDep_TEPC),np.max(eDep_TEPC), eDep_TEPC.size
print np.min(eDep_ACD),np.median(eDep_ACD),np.max(eDep_ACD), eDep_ACD.size

##########################################################################################
# Plot of energy deposited in TEPC 

# Fix the number of bins
numBins = 5E2

# Create histogram
fig_eDep, ax_eDep = plt.subplots(1,1)

# Draw histogram
H = ax_eDep.hist2d(eDep_TEPC,eDep_ACD, numBins, range=np.array([(1E2, 1E3), (1E5, 1E7)]), 
                   cmap=brewer2mpl.get_map('RdPu', 'sequential', 9, reverse=False).mpl_colormap)

# Draw the color bar
cbar = fig_eDep.colorbar(H[3], ax=ax_eDep, spacing='uniform', label='Number of Events')
cbar.solids.set_edgecolor("face")

# Figure Properties
ax_eDep.set_xlabel(r'$\Delta$E in TEPC (eV)')
ax_eDep.set_ylabel(r'$\Delta$E in ACD (eV)')   
ax_eDep.set_xlim(np.min(H[1]), np.max(H[1]))
ax_eDep.set_ylim(np.min(H[2]), np.max(H[2]))

from matplotlib.ticker import ScalarFormatter
ax_eDep.xaxis.set_major_formatter(ScalarFormatter())
ax_eDep.yaxis.set_major_formatter(ScalarFormatter())

# Tight Layout
plt.tight_layout()

# Save the figure 
file_eDep = DataFilename.split('.')[0][:-1]+'_eDep.pdf'
plt.savefig(DataFolder + file_eDep, bbox_inches="tight")
print 'Plot saved to: ' + (DataFolder+file_eDep)

# Show the figure
plt.show(fig_eDep)